In [1]:
import pandas as pd
import numpy as np
import math
import itertools
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1.inset_locator import (inset_axes, InsetPosition, mark_inset)
import math
import copy
import re
from collections import OrderedDict 
import Bio
from Bio import SeqIO
import xls_analysis_classes
from xls_analysis_classes import *

#from xls_analysis_classes import *

matplotlib.rcParams.update({'font.size': 10})

In [3]:
d = pd.read_csv('./proteasome-tri-crosslinks.csv', sep = ',')
data = pd.DataFrame(d)
#data.insert(2, 'dummy A', 'prot1')
#data.insert(3, 'dummy XLA', 'K1')
data.insert(2, 'Subunit DB', 'prot1')
data.insert(3, 'XL DB', 'K1')
#
data.insert(6, 'dummy B', 'prot1')
data.insert(7, 'dummy XLB', 'K1')
data.insert(10, 'dummy C', 'prot1')
data.insert(11, 'dummy XLC', 'K1')
#print(data)
df1 = data.iloc[:, :4].join(data.iloc[:, 12:13]).join(data.iloc[:, 15:16]).join(data.iloc[:, 18:32]).join(data.iloc[:, 46:53])
df1.insert(6, 'Common/Gene Name B', 'prot1')
print(df1.columns)
df2 = data.iloc[:, 4:8].join(data.iloc[:, 13:14]).join(data.iloc[:, 16:17]).join(data.iloc[:, 18:25]).join(data.iloc[:, 32:39]).join(data.iloc[:, 46:53])
#print(df2.columns)
df3 = data.iloc[:, 8:12].join(data.iloc[:, 14:15]).join(data.iloc[:, 17:18]).join(data.iloc[:, 18:25]).join(data.iloc[:, 39:46]).join(data.iloc[:, 46:53])
#print(df3.columns)
df1.to_csv('proteasome-tri-crosslinks-1.csv', index = False)
df2.to_csv('proteasome-tri-crosslinks-2.csv', index = False)
df3.to_csv('proteasome-tri-crosslinks-3.csv', index = False)
#print(df3)

Index(['Subunit A', 'XL A', 'Subunit DB', 'XL DB', 'Subunit Name A',
       'Common/Gene Name A', 'Common/Gene Name B', 'XL Scan', 'XL RT',
       'XL m/z', 'XL Charge', 'XL PPM', 'Corr. XL PPM', 'Corr. XL Isotope',
       'm/z A', 'Charge A', 'Sequence A', 'Protein Mods A', 'PPM A', 'Score A',
       'EV A', 'Ambiguous', 'Redundant', 'Subunit Names', 'Linkage Info',
       'Redundant Count', 'Unique Count', 'CLMS-CSV'],
      dtype='object')


In [11]:
# open a file which is down two directories and called "definitions.txt" and read it
# open this file as a pandas dataframe and name the columns as genename, genericname
heads = ['genename', 'genericname']
dnames = pd.read_csv('../../definitions.txt', header=None, names=heads)
print(dnames)

     Rpt3           6B
0    Rpt4          10B
1    Rpt6            8
2    Rpt2            4
3    Rpt5           6A
4    Rpt1            7
5    Rpn6    non-ATP 2
6   Rpn11   non-ATP 14
7    Rpn7    non-ATP 6
8    Rpn3    non-ATP 3
9    Rpn8    non-ATP 7
10   Rpn2    non-ATP 1
11   Rpn5   non-ATP 12
12   Rpn9   non-ATP 13
